<a href="https://colab.research.google.com/github/linorp/NLP-Project/blob/main/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Create Full Data - No slicing**

In [ ]:
!pip install json_lines

In [ ]:
import json_lines
import json
import pandas as pd

def create_clusters_format(clusters):
  clusters_format = []
  for i in range(len(clusters)):
    for j in range(len(clusters[i])):
      clusters[i][j].append(True)
  return clusters

def create_clusters_format_overlapping(clusters_format):
  overlapping_indexes = set()
  for i in range(len(clusters_format)):
    for j in range(len(clusters_format[i])):
      for t in range(clusters_format[i][j][0], clusters_format[i][j][1]+1):
        if t in overlapping_indexes:
          clusters_format[i][j][2] = False
          break
        else:
          overlapping_indexes.add(t)
  return clusters_format

def map_word_to_cluster(clusters_format):
  d = {}
  for i in range(len(clusters_format)):
    for j in range(len(clusters_format[i])):
      if clusters_format[i][j][2] == False:
        continue
      for index in range(clusters_format[i][j][0], clusters_format[i][j][1]+1):
        assert index not in d
        d[index]=i
  return d

def parse_json_line(line):
  #data = json.loads(line.strip())
  clusters = line['clusters']
  clusters_format = create_clusters_format(clusters)
  clusters_format = create_clusters_format_overlapping(clusters_format)
  map_words = map_word_to_cluster(clusters_format)
  # print(clusters_format)
  # print(map_words)
  word_index = 0
  input= ""
  output = ""

  sentences = line['sentences']
  # print(sentences)
  for i in range(len(sentences)):
    for j in range(len(sentences[i])):
      if word_index in map_words:
        cluster = str(map_words.get(word_index))
      else:
        cluster = "-"
      input += sentences[i][j] + " "
      output += cluster + " "
      word_index += 1
  return input, output

def parse_input_file(pathInput,pathOutput,type):
  output_file = []
  with open (pathInput, 'r') as file:
    count = 0
    for line in json_lines.reader(file):
      input, output = parse_json_line(line)
      output_file.append([input, output])
      count += 1
      # if count == 1000:
      #   break
  print(count)
  output_file_panda = pd.DataFrame(output_file, columns=['input', 'output'])
  if type == 'T5':
    output_file_panda['input'] = "coref: " + output_file_panda['input']
  output_file_panda.to_csv(pathOutput,index=False)

def main():
  output_path_train_T5 = '/content/output/train_T5_format.csv'
  output_path_test_T5 = '/content/output/test_T5_format.csv'
  output_path_dev_T5 = '/content/output/dev_T5_format.csv'

  output_path_train_Bart = '/content/output/train_Bart_format.csv'
  output_path_test_Bart = '/content/output/test_Bart_format.csv'
  output_path_dev_Bart = '/content/output/dev_Bart_format.csv'

  input_path_train = '/content/input/train.jsonlines'
  input_path_test = '/content/input/test.jsonlines'
  input_path_dev = '/content/input/dev.jsonlines'

  type = 'Bart'
  #type = 'T5'
  if type == 'T5':
    parse_input_file(input_path_train,output_path_train_T5,type)
    parse_input_file(input_path_test,output_path_test_T5,type)
    parse_input_file(input_path_dev,output_path_dev_T5,type)
  else:
    parse_input_file(input_path_train,output_path_train_Bart,type)
    parse_input_file(input_path_test,output_path_test_Bart,type)
    parse_input_file(input_path_dev,output_path_dev_Bart,type)

if __name__ == '__main__':
  main()

### **Create Data With Slicing**

In [ ]:
!pip install json_lines

In [ ]:
import json_lines
import json
import pandas as pd

def create_clusters_format(clusters, a):
  clusters_format = []
  for i in range(len(clusters)):
    for j in range(len(clusters[i])):
      if a == 0:
        clusters[i][j].append(True)
  return clusters

def create_clusters_format_overlapping(clusters_format):
  overlapping_indexes = set()
  for i in range(len(clusters_format)):
    for j in range(len(clusters_format[i])):
      for t in range(clusters_format[i][j][0], clusters_format[i][j][1]+1):
        if t in overlapping_indexes:
          clusters_format[i][j][2] = False
          break
        else:
          overlapping_indexes.add(t)
  return clusters_format

def map_word_to_cluster(clusters_format, start_index, end_index):
  d = {}
  clu_index = {}
  count = 0
  count_words = 0
  for i in range(len(clusters_format)):
    for j in range(len(clusters_format[i])):
      if clusters_format[i][j][2] == False:
        continue
      if clusters_format[i][j][0] >= start_index & clusters_format[i][j][0] <= end_index:
        count_words +=1
        if count_words >= 2:
          #assert j not in clu_index:
          clu_index[i] = count
          count += 1
          count_words = 0
          break

  for i in range(len(clusters_format)):
    for j in range(len(clusters_format[i])):
      if i in clu_index:
        for index in range(clusters_format[i][j][0], clusters_format[i][j][1]+1):
          #assert index not in d
          d[index]=clu_index[i]
  return d

def parse_json_line(line, start, end):
  #data = json.loads(line.strip())
  word_index = 0
  word_end = 0
  sentences = line['sentences']
  for i in range(start):
    for j in range(len(sentences[i])):
      word_index += 1
  
  for i in range(end):
    for j in range(len(sentences[i])):
      word_end += 1

  clusters = line['clusters']
  
  clusters_format = create_clusters_format(clusters, start)
  if (start == 0):
    clusters_format = create_clusters_format_overlapping(clusters_format)
  map_words = map_word_to_cluster(clusters_format, word_index, word_end)

  input= ""
  output = ""

  count = 0
  count_d = {}
  for i in range(start, end):
    for j in range(len(sentences[i])):
      if word_index in map_words:
        if not map_words.get(word_index) in count_d:
          count_d[map_words.get(word_index)] = count
          count += 1
        cluster = str(count_d[map_words.get(word_index)])
      else:
        cluster = "-"
      input += sentences[i][j] + " "
      output += cluster + " "
      word_index += 1
  return input, output

def parse_input_file(pathInput,pathOutput,type):
  output_file = []
  count = 0
  with open (pathInput, 'r') as file:
    for line in json_lines.reader(file):
      flag = 0
      a = 0
      #print(len(line["sentences"]))
      while a < (len(line["sentences"])):
        if (a+5 < len(line["sentences"])):
          input, output = parse_json_line(line, a, a+5)
        else:
          input, output = parse_json_line(line, a, len(line["sentences"]))
        #print(input)
        #print(output)
        output_file.append([input, output])
        a += 5
        count += 1
        if count == 50:
          flag = 1
          break 
      if flag == 1:
        break
  
  output_file_panda = pd.DataFrame(output_file, columns=['input', 'output'])
  if type == 'T5':
    output_file_panda['input'] = "coref: " + output_file_panda['input']
  output_file_panda.to_csv(pathOutput,index=False)

def main():
  output_path_train_T5 = '/content/output/train_T5_format.csv'
  output_path_test_T5 = '/content/output/test_T5_format.csv'
  output_path_dev_T5 = '/content/output/dev_T5_format.csv'

  output_path_train_Bart = '/content/output/train_Bart_format.csv'
  output_path_test_Bart = '/content/output/test_Bart_format.csv'
  output_path_dev_Bart = '/content/output/dev_Bart_format.csv'

  input_path_train = '/content/input/train.jsonlines'
  input_path_test = '/content/input/test.jsonlines'
  input_path_dev = '/content/input/dev.jsonlines'

  #type = 'Bart'
  type = 'T5'
  if type == 'T5':
    parse_input_file(input_path_train,output_path_train_T5,type)
    parse_input_file(input_path_test,output_path_test_T5,type)
    parse_input_file(input_path_dev,output_path_dev_T5,type)
  else:
    parse_input_file(input_path_train,output_path_train_Bart,type)
    parse_input_file(input_path_test,output_path_test_Bart,type)
    parse_input_file(input_path_dev,output_path_dev_Bart,type)

if __name__ == '__main__':
  main()

### **Create Data - With Augmentation**

In [ ]:
!pip install json_lines

In [ ]:
import json_lines
import json
import pandas as pd

def create_clusters_format(clusters, a):
  clusters_format = []
  for i in range(len(clusters)):
    for j in range(len(clusters[i])):
      if a == 0:
        clusters[i][j].append(True)
  return clusters

def create_clusters_format_overlapping(clusters_format):
  overlapping_indexes = set()
  for i in range(len(clusters_format)):
    for j in range(len(clusters_format[i])):
      for t in range(clusters_format[i][j][0], clusters_format[i][j][1]+1):
        if t in overlapping_indexes:
          clusters_format[i][j][2] = False
          break
        else:
          overlapping_indexes.add(t)
  return clusters_format

def map_word_to_cluster(clusters_format, start_index):
  d = {}
  clu_index = {}
  count = 0
  count_words = 0
  for i in range(len(clusters_format)):
    for j in range(len(clusters_format[i])):
      if clusters_format[i][j][2] == False:
        continue
      if clusters_format[i][j][0] >= start_index:
        count_words +=1
        if count_words >= 2:
          #assert j not in clu_index:
          clu_index[i] = count
          count += 1
          count_words = 0
          break

  for i in range(len(clusters_format)):
    for j in range(len(clusters_format[i])):
      if i in clu_index:
        for index in range(clusters_format[i][j][0], clusters_format[i][j][1]+1):
          #assert index not in d
          d[index]=clu_index[i]
  return d

def parse_json_line(line, a):
  #data = json.loads(line.strip())
  word_index = 0
  sentences = line['sentences']
  for i in range(a):
    for j in range(len(sentences[i])):
      word_index += 1

  clusters = line['clusters']
  
  clusters_format = create_clusters_format(clusters, a)
  if (a == 0):
    clusters_format = create_clusters_format_overlapping(clusters_format)
  map_words = map_word_to_cluster(clusters_format, word_index)

  input= ""
  output = ""

  count = 0
  count_d = {}
  for i in range(a, len(sentences)):
    for j in range(len(sentences[i])):
      if word_index in map_words:
        if not map_words.get(word_index) in count_d:
          count_d[map_words.get(word_index)] = count
          count += 1
        cluster = str(count_d[map_words.get(word_index)])
      else:
        cluster = "-"
      input += sentences[i][j] + " "
      output += cluster + " "
      word_index += 1
  return input, output

def parse_input_file(pathInput,pathOutput,type):
  output_file = []
  count = 0
  with open (pathInput, 'r') as file:
    for line in json_lines.reader(file):
      for a in range(len(line["sentences"])):
        input, output = parse_json_line(line, a)
        output_file.append([input, output])
        count +=1
  print(count)
  
  output_file_panda = pd.DataFrame(output_file, columns=['input', 'output'])
  if type == 'T5':
    output_file_panda['input'] = "coref: " + output_file_panda['input']
  output_file_panda.to_csv(pathOutput,index=False)

def main():
  output_path_train_T5 = '/content/output/train_T5_format.csv'
  output_path_test_T5 = '/content/output/test_T5_format.csv'
  output_path_dev_T5 = '/content/output/dev_T5_format.csv'

  output_path_train_Bart = '/content/output/train_Bart_format.csv'
  output_path_test_Bart = '/content/output/test_Bart_format.csv'
  output_path_dev_Bart = '/content/output/dev_Bart_format.csv'

  input_path_train = '/content/input/train.jsonlines'
  input_path_test = '/content/input/test.jsonlines'
  input_path_dev = '/content/input/dev.jsonlines'

  type = 'Bart'
  #type = 'T5'
  if type == 'T5':
    parse_input_file(input_path_train,output_path_train_T5,type)
    parse_input_file(input_path_test,output_path_test_T5,type)
    parse_input_file(input_path_dev,output_path_dev_T5,type)
  else:
    parse_input_file(input_path_train,output_path_train_Bart,type)
    parse_input_file(input_path_test,output_path_test_Bart,type)
    parse_input_file(input_path_dev,output_path_dev_Bart,type)

if __name__ == '__main__':
  main()

### **T5 - model**

In [ ]:
# install libraries
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install rich[jupyter]

In [ ]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

# training logger to log training progress
training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)

# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

class YourDataSetClass(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model

    """

    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        """returns the length of dataframe"""

        return len(self.target_text)

    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""

        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }

def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    train_losses = 0.0
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]
        train_losses += loss

        if _ % 5000 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    average_train_loss = train_losses / len(loader)
    return average_train_loss

def validate_on_dev(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.eval()
    dev_losses = 0.0
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data["target_ids"].to(device, dtype=torch.long)
            y_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            ids = data["source_ids"].to(device, dtype=torch.long)
            mask = data["source_mask"].to(device, dtype=torch.long)

            outputs = model(
                input_ids=ids,
                attention_mask=mask,
                decoder_input_ids=y_ids,
                labels=lm_labels,
            )
            loss = outputs[0]
            dev_losses += loss

            if _ % 10000 == 0:
                training_logger.add_row(str(epoch), str(_), str(loss))
                console.print(training_logger)

    average_dev_loss = dev_losses / len(loader)
    return average_dev_loss


def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%1000==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals   

def T5Trainer(
    dataframe, source_text, target_text, model_params, output_dir="./outputs/"
):

    """
    T5 trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"],dropout=0.3)

    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = model.to(device)

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation.
    #train_size = 0.8
    #train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    #val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    #train_dataset = train_dataset.reset_index(drop=True)
    train_dataset = dataframe

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"DEV Dataset: {dev_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = YourDataSetClass(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = YourDataSetClass(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    dev_set = YourDataSetClass(
        dev_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }
    dev_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)
    dev_loader = DataLoader(dev_set, **dev_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )

    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train_loss = train(epoch, tokenizer, model, device, training_loader, optimizer)
        # TODO: add dev loss evaluate + report
        dev_loss = validate_on_dev(epoch, tokenizer, model, device, dev_loader, optimizer)
        console.log(f"epoch {epoch}, training loss: {train_loss}, dev loss: {dev_loss}")

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "predictions.csv"))

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")       


# 1) Change model to t5-base
# 2) Train epochs: 7/8
# 3) Split the data (documents) every X sentences
# Goal: generated.csv, outputs/predictions.csv

model_params = {
    "MODEL": "google/t5-v1_1-base",  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 1,  # training batch size
    "VALID_BATCH_SIZE": 1,  # validation batch size
    "TRAIN_EPOCHS": 3,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 1e-4,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 512,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

train_path = '/content/output/train_T5_format.csv'
dev_path = '/content/output/dev_T5_format.csv'
test_path = '/content/output/test_T5_format.csv'

df = pd.read_csv(train_path)
dev_dataset = pd.read_csv(dev_path)
val_dataset = pd.read_csv(test_path)
# TODO: check that data is read correctly (with commas in the input)

# T5 accepts prefix of the task to be performed:

T5Trainer(
    dataframe=df,
    source_text="input",
    target_text="output",
    model_params=model_params,
    output_dir="/content/outputsT5/",
)

### **Bart - model**

In [ ]:
# install libraries
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install rich[jupyter]

In [ ]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the BART modules from huggingface/transformers
from transformers import BartTokenizer, BartForConditionalGeneration


# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

# training logger to log training progress
training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)

# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

class YourDataSetClass(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model

    """

    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        """returns the length of dataframe"""

        return len(self.target_text)

    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""

        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }

def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    train_losses = 0.0
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]
        train_losses += loss

        if _ % 10 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    average_train_loss = train_losses / len(loader)
    return average_train_loss

def validate_on_dev(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.eval()
    dev_losses = 0.0
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data["target_ids"].to(device, dtype=torch.long)
            y_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            ids = data["source_ids"].to(device, dtype=torch.long)
            mask = data["source_mask"].to(device, dtype=torch.long)

            outputs = model(
                input_ids=ids,
                attention_mask=mask,
                decoder_input_ids=y_ids,
                labels=lm_labels,
            )
            loss = outputs[0]
            dev_losses += loss

            # if _ % 10 == 0:
            #     training_logger.add_row(str(epoch), str(_), str(loss))
            #     console.print(training_logger)

    average_dev_loss = dev_losses / len(loader)
    return average_dev_loss


def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=False)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals   

def BARTTrainer(
    dataframe, source_text, target_text, model_params, output_dir="./outputs/"
):

    """
    BART trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = BartTokenizer.from_pretrained(model_params["MODEL"],dropout=0.5)

    # Defining the model. We are using bart-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = BartForConditionalGeneration.from_pretrained(model_params["MODEL"],dropout=0.5)
    model = model.to(device)

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation.
    #train_size = 0.8
    #train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    #val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    #train_dataset = train_dataset.reset_index(drop=True)
    train_dataset = dataframe

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"Dev Dataset: {dev_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = YourDataSetClass(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = YourDataSetClass(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    dev_set = YourDataSetClass(
        dev_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }
    dev_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)
    dev_loader = DataLoader(dev_set, **dev_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )

    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train_loss = train(epoch, tokenizer, model, device, training_loader, optimizer)
        # TODO: add dev loss evaluate + report
        dev_loss = validate_on_dev(epoch, tokenizer, model, device, dev_loader, optimizer)
        console.log(f"epoch {epoch}, training loss: {train_loss}, dev loss: {dev_loss}")

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "predictions.csv"))

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")       


# 1) Change model to facebook/bart-base
# 2) Train epochs: 7/8
# 3) Split the data (documents) every X sentences
# Goal: generated.csv, outputs/predictions.csv

model_params = {
    "MODEL": "facebook/bart-base",  # model_type:
    "TRAIN_BATCH_SIZE": 1,  # training batch size
    "VALID_BATCH_SIZE": 1,  # validation batch size
    "TRAIN_EPOCHS": 6,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 1e-4,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 512,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

train_path = '/content/output/train_Bart_format.csv'
dev_path = '/content/output/dev_Bart_format.csv'
test_path = '/content/output/test_Bart_format.csv'

df = pd.read_csv(train_path)
dev_dataset = pd.read_csv(dev_path)
val_dataset = pd.read_csv(test_path)

# TODO: check that data is read correctly (with commas in the input)

# T5 accepts prefix of the task to be performed:

BARTTrainer(
    dataframe=df,
    source_text="input",
    target_text="output",
    model_params=model_params,
    output_dir="/content/outputsBART/",
)

### **Bart No Pretraining - model**

In [ ]:
# install libraries
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install rich[jupyter]

In [ ]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the BART modules from huggingface/transformers
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig


# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

# training logger to log training progress
training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)

# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

class YourDataSetClass(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model

    """

    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        """returns the length of dataframe"""

        return len(self.target_text)

    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""

        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }

def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    train_losses = 0.0
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]
        train_losses += loss

        if _ % 10 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    average_train_loss = train_losses / len(loader)
    return average_train_loss

def validate_on_dev(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.eval()
    dev_losses = 0.0
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data["target_ids"].to(device, dtype=torch.long)
            y_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            ids = data["source_ids"].to(device, dtype=torch.long)
            mask = data["source_mask"].to(device, dtype=torch.long)

            outputs = model(
                input_ids=ids,
                attention_mask=mask,
                decoder_input_ids=y_ids,
                labels=lm_labels,
            )
            loss = outputs[0]
            dev_losses += loss

            if _ % 10 == 0:
                training_logger.add_row(str(epoch), str(_), str(loss))
                console.print(training_logger)

    average_dev_loss = dev_losses / len(loader)
    return average_dev_loss


def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals   

def BARTTrainer(
    dataframe, source_text, target_text, model_params, output_dir="./outputs/"
):

    """
    BART trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = BartTokenizer.from_pretrained(model_params["MODEL"])

    # Defining the model. We are using bart-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    config = BartConfig(dropout=0.1)
    model = BartForConditionalGeneration(config)
    model = model.to(device)

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation.
    #train_size = 0.8
    #train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    #val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    #train_dataset = train_dataset.reset_index(drop=True)
    train_dataset = dataframe

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"Dev Dataset: {dev_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = YourDataSetClass(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = YourDataSetClass(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    dev_set = YourDataSetClass(
        dev_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }
    dev_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)
    dev_loader = DataLoader(dev_set, **dev_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )

    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train_loss = train(epoch, tokenizer, model, device, training_loader, optimizer)
        # TODO: add dev loss evaluate + report
        dev_loss = validate_on_dev(epoch, tokenizer, model, device, dev_loader, optimizer)
        console.log(f"epoch {epoch}, training loss: {train_loss}, dev loss: {dev_loss}")

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "predictions.csv"))

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")       


# 1) Change model to facebook/bart-base
# 2) Train epochs: 7/8
# 3) Split the data (documents) every X sentences
# Goal: generated.csv, outputs/predictions.csv

model_params = {
    "MODEL": "facebook/bart-base",  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 1,  # training batch size
    "VALID_BATCH_SIZE": 1,  # validation batch size
    "TRAIN_EPOCHS": 3,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 1e-4,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 512,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

train_path = '/content/output/train_Bart_format.csv'
dev_path = '/content/output/dev_Bart_format.csv'
test_path = '/content/output/test_Bart_format.csv'

df = pd.read_csv(train_path)
dev_dataset = pd.read_csv(dev_path)
val_dataset = pd.read_csv(test_path)




# TODO: check that data is read correctly (with commas in the input)

# T5 accepts prefix of the task to be performed:

BARTTrainer(
    dataframe=df,
    source_text="input",
    target_text="output",
    model_params=model_params,
    output_dir="/content/outputsBartNoPre/",
)

### **From predictions to jsonlines**


In [ ]:
!pip install jsonlines


In [ ]:
import csv
import json
import jsonlines


def convert(inputpath):
  with jsonlines.open('generate.JSONLINES', mode='w') as generateW:
      with jsonlines.open('actual.JSONLINES', mode='w') as actualW:
        with open(inputpath) as f:
            reader = csv.reader(f)
            count = 0 

            for row in reader:
              
              start_index = 0
              end_index = 0

              actual = row[2]
              actuals=""
              max = -1
              for i in range(len(actual)):
                if actual[i] != " ":
                  actuals+=actual[i]
                if (actual[i]).isdigit():
                  if max < int(actual[i]):
                    max = int(actual[i])
              #print(actuals)
              actual_clusters = []
              if max != -1:
                for i in range(max + 1):
                  actual_clusters.append([])
                actual_index = 0
                while actual_index < len(actuals):
                  if actuals[actual_index].isdigit():
                    start_index = actual_index
                    end_index = actual_index
                    while (end_index+1) < len(actuals):
                      if actuals[start_index] == actuals[end_index+1]:
                        end_index += 1
                      else:
                        break
                    actual_clusters[int(actuals[actual_index])].append([start_index, end_index])
                    actual_index = end_index + 1
                  else:
                    actual_index += 1
            
              #print(actual_clusters)
              actualW.write(tuple(actual_clusters))

              generate = row[1]
              generates=""
              max = -1
              for i in range(len(generate)):
                if generate[i] != " ":
                  generates+=generate[i]
                if (generate[i]).isdigit():
                  if max < int(generate[i]):
                    max = int(generate[i])
              #print(generates)
        
              generate_clusters = []
              count = 0
              if max != -1:
                for i in range(max + 1):
                  generate_clusters.append([])
                generate_index = 0
                while generate_index < len(generates):
                  if generates[generate_index].isdigit():
                    start_index = generate_index
                    end_index = generate_index
                    while (end_index+1) < len(generates):
                      if generates[start_index] == generates[end_index+1]:
                        end_index += 1
                      else:
                        break
                    generate_clusters[int(generates[generate_index])].append(tuple([start_index, end_index]))
                    generate_index = end_index + 1
                  else:
                    generate_index += 1
              #print(generate_clusters)
              generate_clusters = tuple(generate_clusters)
              generateW.write(generate_clusters)

def main():
  inputpath = '/content/predictions.csv'
  convert(inputpath)
  # convert()

if __name__ == '__main__':
  main()


### **MUC & B3 & CEAF** - **F1 , Recall , Precision**

In [ ]:
!pip install jsonlines
!pip install rich[jupyter]

In [ ]:
import jsonlines
from collections import Counter
import numpy as np
from scipy.optimize import linear_sum_assignment
from rich.table import Column, Table
from rich.console import Console
from rich import box


def extract_mentions_to_predicted_clusters_from_clusters(num_cluster,actual_clusters):
    mention_to_gold = {}
    for gc in range(num_cluster):
        for mention in actual_clusters[gc]:
            mention_to_gold[tuple(mention)] = str(gc)
    return mention_to_gold

def phi4(c1, c2):
    return 0 if len(c1) + len(c2) == 0 else 2 * len([m for m in c1 if m in c2]) / float(len(c1) + len(c2))

def ceafe(clusters, gold_clusters):
    clusters = [c for c in clusters if len(c) != 1]
    scores = np.zeros((len(gold_clusters), len(clusters)))
    for i in range(len(gold_clusters)):
        for j in range(len(clusters)):
            scores[i, j] = phi4(gold_clusters[i], clusters[j])
    row_ind, col_ind = linear_sum_assignment(-scores)
    similarity = sum(scores[row_ind, col_ind])
    return similarity, len(clusters), similarity, len(gold_clusters)


def b_cubed(clusters, mention_to_gold):
  num, dem = 0, 0
  for c in clusters:
    if len(c) == 1:
      continue
    gold_counts = Counter()
    correct = 0
    for m in c:
      m = tuple(m)
      if m in mention_to_gold:
        gold_counts[tuple(mention_to_gold[m])] += 1
        for c2, count in gold_counts.items():
          if len(c2) != 1:
            correct += count * count
        num += correct / float(len(c))
        dem += len(c)
  return num, dem

def muc(clusters, mention_to_gold):
    tp, p = 0, 0
    for c in clusters:
      p += len(c) - 1
      tp += len(c)
      linked = set()
      for m in c:
        m = tuple(m)
        if m in mention_to_gold:
            linked.add(mention_to_gold[m])
        else:
            tp -= 1
      tp -= len(linked)
    return tp, p


def get_f1(p_num , p_den , r_num, r_den,beta=1 ):
  p = 0 if p_den == 0 else p_num / float(p_den)
  r = 0 if r_den == 0 else r_num / float(r_den)
  return 0 if p + r == 0 else (1 + beta * beta) * p * r / (beta * beta * p + r)

def get_recall(p_num , p_den , r_num, r_den):
  return 0 if r_num == 0 else r_num / float(r_den)

def get_precision(p_num , p_den , r_num, r_den):
  return 0 if p_num == 0 else p_num / float(p_den)



def calc_4_values(type, actuals, predictions):

  p_num = 0
  p_den = 0
  r_num = 0
  r_den = 0

  for i in range(len(actuals)):
    line_actual = actuals[i]
    line_predicted = predictions[i]
    # if len(line_actual) > 0:
    num_cluster = len(line_actual)
    mention_to_gold = extract_mentions_to_predicted_clusters_from_clusters(num_cluster,line_actual)
    # print(mention_to_gold)
    num_cluster = len(line_predicted)
    mention_to_predicted = extract_mentions_to_predicted_clusters_from_clusters(num_cluster,line_predicted)
    # print(mention_to_predicted)

    if type == 'ceafe':
      pn , pd , rn, rd =  ceafe(line_predicted, line_actual)
    elif type == 'muc':
      pn , pd =  muc(line_predicted, mention_to_gold)
      rn , rd =  muc(line_actual, mention_to_predicted)
    else:
      pn , pd =  b_cubed(line_predicted, mention_to_gold)
      rn , rd =  b_cubed(line_actual, mention_to_predicted)
    
    p_num += pn
    p_den += pd
    r_num += rn
    r_den += rd

  return p_num , p_den, r_num, r_den


def main():

  Title = 'Results for model T5 with the parameters:' + '\n' + ' - learning rate = 1e-5' + '\n' + ' - Batch size = 1' + '\n' + ' - Number of Epochs = 6' + '\n' + ' - Number of Beam search = 2' 
  console = Console()
  table_1 = Table(
        Column("Model", justify="center"),
        Column("Metric", justify="center"),
        Column("F1", justify="center"),
        Column("Recall", justify="center"),
        Column("Precision", justify="center"),
        title=Title,
        pad_edge=False,
        box=box.ASCII,
  )

  path_actual = '/content/actual.JSONLINES'
  path_generate = '/content/generate.JSONLINES'
  actuals = []
  predictions = []
  with jsonlines.open(path_actual) as reader:
    for obj in reader:
      actuals.append(obj)
  with jsonlines.open(path_generate) as reader:
    for obj in reader:
      predictions.append(obj)

  p_num_muc , p_den_muc, r_num_muc, r_den_muc  = calc_4_values('muc',actuals, predictions)

  f1_muc = get_f1(p_num_muc , p_den_muc, r_num_muc, r_den_muc,beta=1)
  recall_muc = get_recall(p_num_muc , p_den_muc, r_num_muc, r_den_muc)
  precision_muc = get_precision(p_num_muc , p_den_muc, r_num_muc, r_den_muc)
  #print (f1_muc , recall_muc , precision_muc)
  table_1.add_row('Bart', 'MUC' ,str(f1_muc), str(recall_muc),str(precision_muc))


  p_num_B3 , p_den_B3, r_num_B3, r_den_B3  = calc_4_values(None,actuals, predictions)

  f1_B3 = get_f1(p_num_B3 , p_den_B3, r_num_B3, r_den_B3,beta=1)
  recall_B3 = get_recall(p_num_B3 , p_den_B3, r_num_B3, r_den_B3)
  precision_B3 = get_precision(p_num_B3 , p_den_B3, r_num_B3, r_den_B3)
  #print (f1_B3 , recall_B3 , precision_B3)
  table_1.add_row('Bart', 'B3' ,str(f1_B3), str(recall_B3),str(precision_B3))


  p_num_CAEF , p_den_CAEF, r_num_CAEF, r_den_CAEF  = calc_4_values('ceafe',actuals, predictions)

  f1_CAEF = get_f1(p_num_CAEF , p_den_CAEF, r_num_CAEF, r_den_CAEF,beta=1)
  recall_CAEF = get_recall(p_num_CAEF , p_den_CAEF, r_num_CAEF, r_den_CAEF)
  precision_CAEF = get_precision(p_num_CAEF , p_den_CAEF, r_num_CAEF, r_den_CAEF)
  #print (f1_CAEF , recall_CAEF , precision_CAEF)
  table_1.add_row('Bart', 'CAEF' ,str(f1_CAEF), str(recall_CAEF),str(precision_CAEF))

  console.print(table_1)

  table_2 = Table(
        Column("Model", justify="center"),
        Column("Avg_F1", justify="center"),
        Column("Avg_Recall", justify="center"),
        Column("Avg_Precision", justify="center"),
        title="Averages",
        pad_edge=False,
        box=box.ASCII,
  )

  F1_avg = (f1_muc + f1_B3 + f1_CAEF) / 3
  Recall_avg = (recall_muc + recall_B3 + recall_CAEF) / 3
  Precision_avg = (precision_muc + precision_B3 + precision_CAEF) / 3
  table_2.add_row('Bart', str(F1_avg), str(Recall_avg),str(Precision_avg))
  console.print(table_2)

if __name__ == '__main__':
  main()
